In [ ]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Concatenate
from tensorflow.keras.models import Model

def load_image(image_path, target_size):
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    plt.imshow(image)# Load image in color
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, target_size)
    image = image.astype(np.float32) / 255.0  # Normalize pixel values
    return image

def load_data(data_folder, target_size):
    left_images = []
    right_images = []
    middle_images = []

    for sample_folder in os.listdir(data_folder):
        sample_folder_path = os.path.join(data_folder, sample_folder)
        left_frame_path = os.path.join(sample_folder_path, 'left_frame.jpg')
        right_frame_path = os.path.join(sample_folder_path, 'right_frame.jpg')
        middle_frame_path = os.path.join(sample_folder_path, 'middle_frame.jpg')

        left_image = load_image(left_frame_path, target_size)
        right_image = load_image(right_frame_path, target_size)
        middle_image = load_image(middle_frame_path, target_size)

        left_images.append(left_image)
        right_images.append(right_image)
        middle_images.append(middle_image)

    return np.array(left_images), np.array(right_images), np.array(middle_images)

    
def build_model(input_shape):
    left_input = Input(shape=input_shape, name='left_input')
    right_input = Input(shape=input_shape, name='right_input')

    # Shared convolutional layers
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')
    conv7 = Conv2D(256, (3, 3), activation='relu', padding='same')
    left_features = conv3(conv2(conv1(left_input)))
    right_features = conv3(conv2(conv1(right_input)))

    # Concatenate features
    merged_features = Concatenate(axis=-1)([left_features, right_features])

    # Additional convolutional layers
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same')
    conv4 = Conv2D(128, (3, 3), activation='relu', padding='same')
    conv5 = Conv2D(64, (3, 3), activation='relu', padding='same')
    conv6 = Conv2D(32, (3, 3), activation='relu', padding='same')

    output = conv6(conv5(conv4(conv8(merged_features))))

    # Output layer
    output_image = Conv2D(3, (3, 3), activation='sigmoid', padding='same', name='output')(output)

    model = Model(inputs=[left_input, right_input], outputs=output_image)
    return model

# Load data
data_folder = r'C:\Users\papir\Desktop\DSC 550\training_samples'
input_shape = (256, 256, 3)  # Specify the shape of your input images
left_images, right_images, middle_images = load_data(data_folder, input_shape[:2])

# Build model
model = build_model(input_shape)
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit([left_images, right_images], middle_images, batch_size=16, epochs=75, validation_split=0.2)


In [ ]:
import matplotlib.pyplot as plt

# Load some testing images
testing_data_folder = r'C:\Users\papir\Desktop\DSC 550\testing_samples'
testing_left_images, testing_right_images, _ = load_data(testing_data_folder, input_shape[:2])

# Use the model to predict middle images
predicted_middle_images = model.predict([testing_left_images, testing_right_images])

# Visualize some sample results
num_samples = 8
plt.figure(figsize=(15, 10))
for i in range(num_samples):
    # Original left image
    plt.subplot(num_samples, 3, i * 3 + 1)
    plt.imshow(testing_left_images[i])
    plt.title('Left Image')
    plt.axis('off')

    # Original right image
    plt.subplot(num_samples, 3, i * 3 + 2)
    plt.imshow(testing_right_images[i])
    plt.title('Right Image')
    plt.axis('off')

    # Predicted middle image
    plt.subplot(num_samples, 3, i * 3 + 3)
    plt.imshow(predicted_middle_images[i])
    plt.title('Predicted Middle Image')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
model.save('model.h5')

In [ ]:
from zipfile import ZipFile
file_name = '/content/testing_samples/testing_samples.zip'

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')